# Constraint statistics
Explore trends of regional nonsense constraint. 

In [1]:
# Import modules
from pathlib import Path
import os

import numpy as np
import pandas as pd

from src import setup_logger
from src import constants as C

In [2]:
# Logging
logger = setup_logger(Path(os.path.abspath('')).stem)

[22-Nov-23 14:49:17] INFO <module>(): Duplicated by enst / region / csq: 0


Find regions in which nonsense variants are possible, and in which constraint statistics could be calculated.

## Constraint statistics
Get the number of constrained regions

In [11]:
# Masks for constraint criteria
m1 = ~stop.z.isna()
m2 = stop.syn_z_1sd == True
m3 = stop.fdr_p < 0.05
m4 = stop.oe < 0.35
m5 = stop.n_obs == 0

# Cumulative criteria
ma = (m1 & m2).rename("pass_syn_z")
mb = (ma & m3).rename("pass_fdr")
mc = (mb & m4).rename("pass_oe")
md = (mc & m5).rename("none_observed")

stop = pd.concat([stop, ma, mb, mc, md], axis=1)
stop.sample(1)

,enst,region,csq,n_pos,n_obs,n_exp,oe,prop_obs,prop_exp,mu,...,p,fdr_p,pli,loeuf,gnomad_flags,syn_z_1sd,pass_syn_z,pass_fdr,pass_oe,none_observed
48307,ENST00000358465,nmd_target,stop_gained,376.0,11.0,74.275,0.148098,0.029255,0.19754,0.000002,...,2.486296e-16,1.312831e-14,1.0,0.23,[],True,True,True,True,False


In [12]:
# Regions meeting cumulative criteria
stop_valid = stop.groupby("region").agg(
    transcripts=("csq", "count"),
    valid_regions=("oe", "count"),
    valid_statistics=("z", "count"),
    pass_syn_z=("pass_syn_z", "sum"),
    pass_fdr=("pass_fdr", "sum"),
    pass_oe=("pass_oe", "sum"),
    none_observed=("none_observed", "sum"),
)
stop_valid

,transcripts,valid_regions,valid_statistics,pass_syn_z,pass_fdr,pass_oe,none_observed
region,,,,,,,
distal_nmd,19432,18762,9574,6509,1333,862,201
long_exon,19432,3475,2135,1417,633,439,52
nmd_target,19432,16423,14247,7968,3250,1681,137
start_proximal,19432,19161,1607,1252,134,131,96
transcript,19432,19419,19011,10836,4710,1913,90


In [6]:
pd.concat([df_valid, syn_z, fdr_lt_005, oe_lt_035, obs_0], axis=1)

NameError: name 'df_valid' is not defined

In [ ]:
# # ### Spearman rank Z vs LOEUF


# m1 = df["region"] == "transcript"
# m2 = df["csq"] == "nonsense"

# z = df[m1 & m2]["z"]
# loeuf = df[m1 & m2]["loeuf"]

# print(spearmanr(z, loeuf, nan_policy="omit", alternative="two-sided"))